In [1]:
# Geocodes place names using the HERE API

import argparse
import uuid
import pandas as pd
import os
import requests
import csv
import logging
import csv
from csv import writer

In [2]:
#Set Working Directory - Home Computer - NEW IMAC
os.chdir('/Users/davidwhite/Box Sync/Default Sync Folder/Projects/Media_Forensics/Projects/satc/whitedl/domestic_disinfo/working')

In [3]:

# set logger at root level and DEBUG level
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# create a file handler
fh = logging.FileHandler('user_ids_locations.log')
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)

formatter = logging.Formatter('%(asctime)s  %(name)s  %(levelname)s: %(message)s')
fh.setFormatter(formatter)

In [4]:
# Set your output file name here.
output_filename = 'user_ids_locations_out.csv'
# Set your input file here
input_filename = 'locations_demo.csv'
# Specify the column name in your input data that contains addresses here
address_column_name = "user_reported_location"

In [ ]:
# Read the data to a Pandas Dataframe
data = pd.read_csv(input_filename, encoding='utf8')

# Form a list of addresses for geocoding:
# Make a big list of all of the addresses to be processed.
addresses = data[address_column_name].tolist()
print(addresses)


In [6]:
def append_list_as_row(output_filename, output):
    # Open file in append mode
    with open(output_filename, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(output)


In [ ]:
# Read the data to a Pandas Dataframe
data = pd.read_csv(input_filename, encoding='utf8')

# Form a list of addresses for geocoding:
# Make a big list of all of the addresses to be processed.
addresses = data[address_column_name].tolist()
#print(addresses)

# Loop based on the CSV file that contains the locations to be geocoded
iter_len=len(addresses)
for q in range(iter_len):
    # Defines the Rest API
    geocode_url = "https://geocode.search.hereapi.com/v1/geocode?q={}".format(addresses[q]) + "&apikey=YOUR API KEY"
    try:
        results = requests.get(geocode_url)
        #print('THIS IS THE CURRENT URL:', geocode_url), '\n'

    except requests.exceptions.RequestException as err:
        repr(err)

    else:
        results = results.json()
        print('Dictionary Results:',results)
        if 'items' not in results: 
            output = ('NULL','NULL','NULL','NULL',addresses[q])
            #print('NULL OUTPUT:', output), '\n'
            append_list_as_row(output_filename, output)
            continue
        

    if results['items'] == []:
        output = ('NULL','NULL','NULL','NULL',addresses[q])
        #print('NULL OUTPUT:', output), '\n'
        append_list_as_row(output_filename, output)
        continue
                
    else:
        output = (    
        results['items'][0]['position']['lat'],
        results['items'][0]['position']['lng'],
        results['items'][0]['address']['countryCode'],
        results['items'][0]['address']['countryName'],
        addresses[q]
        )

    append_list_as_row(output_filename, output)

# A quick hack to get column labels. 
user_ids_locations_out = pd.read_csv('user_ids_locations_out.csv', header=None, names=['lat','long','countryCode','countryName','user_reported_location'])
user_ids_locations_out.to_csv('user_ids_locations_out.csv')